In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.models import Sequential
from keras.optimizers import SGD,Adam
from skimage.transform import resize
from sklearn.model_selection import train_test_split

# Load data

In [ ]:
X_raw = np.load('X.npy')
X_raw = X_raw / 255.
y = np.load('y.npy')
X = []
size = (64, 64)
for x in X_raw:
    X.append(resize(x, size))
X = np.array(X)

# Show data

In [ ]:
plt.figure(figsize=(20, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    idx = np.random.choice(X.shape[0], 1)[0]
    plt.title('id:{} val:{}'.format(idx, y[idx]))
    plt.imshow(-X[idx], cmap='gray')
    plt.axis('off')
plt.show()

# Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
if X_train.ndim == 3:
    X_train = X_train[..., None]
    X_test = X_test[..., None]
    print(X_train.shape, X_test.shape)

# Create model

In [ ]:
bs = 16
lr = 0.0001

class_mapping = {
    'ma': 0, 'mb': 1, 'mc': 2, 'md': 3, 'me': 4, 'mf': 5, 'mg': 6, 'mh': 7, 'mi': 8, 'mj': 9,
    'mk': 10, 'ml': 11, 'mm': 12, 'mn': 13, 'mo': 14, 'mp': 15, 'mq': 16, 'mr': 17, 'ms': 18, 'mt': 19,
    'mu': 20, 'mv': 21, 'mw': 22, 'mx': 23, 'my': 24, 'mz': 25,
    'ba': 26, 'bb': 27, 'bc': 28, 'bd': 29, 'be': 30, 'bf': 31, 'bg': 32, 'bh': 33, 'bi': 34, 'bj': 35,
    'bk': 36, 'bl': 37, 'bm': 38, 'bn': 39, 'bo': 40, 'bp': 41, 'bq': 42, 'br': 43, 'bs': 44, 'bt': 45,
    'bu': 46, 'bv': 47, 'bw': 48, 'bx': 49, 'by': 50, 'bz': 51
}
y_train = np.array([class_mapping[label] for label in y_train])
y_test = np.array([class_mapping[label] for label in y_test])
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
from keras.src.layers import MaxPooling2D

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Dense(256, activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dense(256, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  
    Dense(52, activation='softmax')  
])

In [ ]:
optimizer1 = Adam(learning_rate=lr)
model.compile(optimizer=optimizer1, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model

In [ ]:
log = model.fit(X_train, y_train, batch_size=bs, epochs=100, validation_data=(X_test, y_test))
model.save('model.keras')

# testing model

In [ ]:
#Probando modelo

etiqueta_a_symbols = {
    0: 'ma', 1: 'mb', 2: 'mc', 3: 'md', 4: 'me', 5: 'mf', 6: 'mg', 7: 'mh', 8: 'mi', 9: 'mj',
    10: 'mk', 11: 'ml', 12: 'mm', 13: 'mn', 14: 'mo', 15: 'mp', 16: 'mq', 17: 'mr', 18: 'ms', 19: 'mt',
    20: 'mu', 21: 'mv', 22: 'mw', 23: 'mx', 24: 'my', 25: 'mz',
    26: 'ba', 27: 'bb', 28: 'bc', 29: 'bd', 30: 'be', 31: 'bf', 32: 'bg', 33: 'bh', 34: 'bi', 35: 'bj',
    36: 'bk', 37: 'bl', 38: 'bm', 39: 'bn', 40: 'bo', 41: 'bp', 42: 'bq', 43: 'br', 44: 'bs', 45: 'bt',
    46: 'bu', 47: 'bv', 48: 'bw', 49: 'bx', 50: 'by', 51: 'bz'
}

In [ ]:
idx = np.random.choice(X_test.shape[0], 1)[0]
im = X_test[idx]
label = y_test[idx]
nombre_symbol = etiqueta_a_symbols.get(label, 'unknown')

In [ ]:
plt.title('(test) id:{} val:{} ({})'.format(idx, label, nombre_symbol))
plt.axis('off')
plt.imshow(-im[:, :, 0], cmap='gray')
plt.show()

In [ ]:
#Usando el modelo
salida = model.predict(im[None, :, :, :])[0]

In [ ]:
plt.figure()
plt.title("Predicción: {} ({}), Label: {}".format(salida.argmax(), nombre_symbol, label))
plt.ylabel("Probabilidad")
plt.xlabel("Clase")
plt.ylim([0, 1])
plt.bar(np.arange(len(salida)), salida)
plt.show()

In [ ]:
predicciones = model.predict(X_test)
etiquetas_predichas = np.argmax(predicciones, axis=1)


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
precision = accuracy_score(y_test, etiquetas_predichas)
print(f'Precisión: {precision}')

In [ ]:
from sklearn.metrics import confusion_matrix

matriz_confusion = confusion_matrix(y_test, etiquetas_predichas)
print('Matriz de Confusión:')
print(matriz_confusion)